In [6]:
import pandas as pd
import numpy as np
import numpy.linalg as la
from sklearn.model_selection import train_test_split
from sklearn.base import clone
import timeit
import math

In [7]:
def pca(F, X):
    n, d = X.shape
    mu = np.zeros((d, 1))
    Z = np.zeros((d, F))
    for i in range(d):
        mu[i] = (1. / n) * np.sum(X[:, [i]])
    X = X - mu.T
    U, s, Vt = la.svd(X, False)
    g = s[:F]
    for i in range(F):
        g[i] = 1. / g[i]
    W = Vt[:F]
    Z = np.dot(W.T, np.diag(g))
    return (mu, Z)

def pca_proj(X,mu,Z):
    n, d = X.shape
    X = X - mu.T
    return np.dot(X, Z)

In [8]:
def k_fold(k, model, f, X, y, error_type="mse"):
    n, d = X.shape
    z = np.zeros((k, 1))
    for i in range(k):
        T = list(range(int((i * n) / k), int((n * (i + 1) / k))))
        S = [j for j in range(n) if j not in T]
        curr_model = clone(model)

        training_mu, training_Z = pca(f, X[S])
        training_X = pca_proj(X[S], training_mu, training_Z)

        curr_model.fit(training_X, y[S])

        test_X = pca_proj(X[T], training_mu, training_Z)

        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        if error_type == "mse":
            z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(test_X)) ** 2)
        elif error_type == "log_mse":
            z[i] = (1. / len(T)) * np.sum((np.log(y[T] + 1) - np.log(curr_model.predict(test_X) + 1)) ** 2)
    return z

In [9]:
def bootstrapping(B, model, f, X, y, error_type="mse"):
    n, d = X.shape
    z = np.zeros((B, 1))
    for i in range(B):
        u = np.random.choice(n, n, replace=True)
        S = np.unique(u)
        T = np.setdiff1d(np.arange(n), S, assume_unique=True)
        curr_model = clone(model)

        training_mu, training_Z = pca(f, X[u])
        training_X = pca_proj(X[u], training_mu, training_Z)

        curr_model.fit(training_X, y[u])

        test_X = pca_proj(X[T], training_mu, training_Z)

        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        # theta_hat will be d by 1
        if error_type == "mse":
            z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(test_X)) ** 2)
        elif error_type == "log_mse":
            z[i] = (1. / len(T)) * np.sum((np.log(y[T] + 1) - np.log(curr_model.predict(test_X) + 1)) ** 2)
    return z

In [10]:
def evaluate_model(model, f, X, y, k=5, B=5):
    ########################KFOLD###################
    print('Evaluating K-fold with %d folds.' % k)
    start_time = timeit.default_timer()
    k_fold_z = k_fold(k, model, f, X, y, error_type="log_mse")
    elapsed = timeit.default_timer() - start_time
    
    k_fold_mse = np.mean(k_fold_z)
    print('K-fold Mean Squared log Error: ', k_fold_mse)
    
    k_fold_rmse = math.sqrt(k_fold_mse)
    print('K-fold Square Root Mean Squared log Error: ', k_fold_rmse)

    print("Time elapsed for k-fold: ", elapsed)
    
    print()
    print()
    ###################BOOTSTRAPPING################
    print('Evaluating bootstrapping with %d bootstraps.' % B)
    start_time = timeit.default_timer()
    bootstrapping_z = bootstrapping(B, model, f, X, y)
    elapsed = timeit.default_timer() - start_time
    
    bootstrapping_mse = np.mean(bootstrapping_z)
    print('Bootstrapping Mean Squared Error: ', bootstrapping_mse)
    
    bootstrapping_rmse = math.sqrt(bootstrapping_mse)
    print('Bootstrapping Square Root Mean Squared Error: ', bootstrapping_rmse)

    print("Time elapsed for bootstrapping: ", elapsed)
    
    return (k_fold_z, k_fold_mse, k_fold_rmse, bootstrapping_z, bootstrapping_mse, bootstrapping_rmse)

# Data Processing

In [11]:
data = pd.read_csv("train.csv", header=0)
print(data.shape)

X = data.iloc[:,:-1]
Y = data.iloc[:,-1:]

print(X.shape)
print(Y.shape)

(1460, 81)
(1460, 80)
(1460, 1)


In [12]:
# this just sums up how many nulls per feature and divides to find percentage of nulls per feature
# if over 50% null then print the feature
data_keys = X.keys()
for i, b in enumerate((X.isnull().sum() / X.shape[0]) > 0.5):
    if b:
        print(data_keys[i])

Alley
PoolQC
Fence
MiscFeature


In [13]:
# categorical columns
catCols = set(list(X))-set(list(X._get_numeric_data()))
print(catCols)

# #TRY dropping all cat cols
# data = data.drop(columns=catCols)

{'Electrical', 'Exterior2nd', 'LandContour', 'Condition2', 'BsmtCond', 'HouseStyle', 'MasVnrType', 'BsmtQual', 'KitchenQual', 'Utilities', 'HeatingQC', 'Heating', 'RoofMatl', 'CentralAir', 'SaleCondition', 'GarageCond', 'LotConfig', 'Neighborhood', 'BldgType', 'Foundation', 'FireplaceQu', 'RoofStyle', 'Condition1', 'ExterQual', 'PavedDrive', 'MiscFeature', 'SaleType', 'MSZoning', 'GarageType', 'BsmtFinType2', 'Exterior1st', 'BsmtFinType1', 'PoolQC', 'GarageQual', 'Fence', 'GarageFinish', 'LotShape', 'Alley', 'Functional', 'LandSlope', 'BsmtExposure', 'Street', 'ExterCond'}


In [14]:
#Perform one hot encoding on all categorical columns
frames = []
for col in catCols:
    oneHot_encoded = pd.get_dummies(X[col])
    oneHot_encoded = oneHot_encoded.add_prefix(col + '_is_')
    frames.append(oneHot_encoded)

X = X.drop(catCols, axis=1)

X = pd.concat(frames, axis=1)

In [15]:
X.keys()

Index(['Electrical_is_FuseA', 'Electrical_is_FuseF', 'Electrical_is_FuseP',
       'Electrical_is_Mix', 'Electrical_is_SBrkr', 'Exterior2nd_is_AsbShng',
       'Exterior2nd_is_AsphShn', 'Exterior2nd_is_Brk Cmn',
       'Exterior2nd_is_BrkFace', 'Exterior2nd_is_CBlock',
       ...
       'BsmtExposure_is_Gd', 'BsmtExposure_is_Mn', 'BsmtExposure_is_No',
       'Street_is_Grvl', 'Street_is_Pave', 'ExterCond_is_Ex',
       'ExterCond_is_Fa', 'ExterCond_is_Gd', 'ExterCond_is_Po',
       'ExterCond_is_TA'],
      dtype='object', length=252)

In [16]:
X.isnull().values.any()

False

In [17]:
# 80:20 train test ratio
test_size = 0.2
# This function splits the training and target sets into random train and test subsets.
# X_train and X_test are subsets of the training data
# y_train and y_test are subsets the the target data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)

# PCA Feature Selection

In [18]:
F = 50
f = 50

In [19]:
X_mu, X_Z = pca(F, X.values)
X_pca = pca_proj(X.values, X_mu, X_Z)

In [20]:
print(X_mu.shape)
print(X_Z.shape)
print(X_pca.shape)

(252, 1)
(252, 50)
(1460, 50)


In [21]:
X_train_mu, X_train_Z = pca(F, X_train.values)

In [22]:
print(X_train_mu.shape)
print(X_train_Z.shape)

(252, 1)
(252, 50)


In [23]:
X_train_pca = pca_proj(X_train.values, X_train_mu, X_train_Z)
print(X_train_pca.shape)

X_test_pca = pca_proj(X_test.values, X_train_mu, X_train_Z)
print(X_test_pca.shape)

(1168, 50)
(292, 50)


# AdaBoost

In [24]:
from sklearn.ensemble import AdaBoostRegressor
adaBoost = AdaBoostRegressor()
k_z, k_mse, k_rmse, b_z, b_mse, b_rmse = evaluate_model(adaBoost, f, X.values, Y.values.ravel(), k=5, B=5)

adaBoost.fit(X_train_pca, y_train.values.ravel())
adaBoost.score(X_test_pca, y_test.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared log Error:  0.08444942076644026
K-fold Square Root Mean Squared log Error:  0.29060182512579
Time elapsed for k-fold:  2.4636729999999716


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  2728630087.457372
Bootstrapping Square Root Mean Squared Error:  52236.29090447916
Time elapsed for bootstrapping:  2.7554159999999683


0.5572909570255471

In [25]:
#View Predicted values
predicted = adaBoost.predict(X_test_pca)
ada_pred = y_test.copy()
ada_pred['predicted'] = predicted
ada_pred.head()

,SalePrice,predicted
665,230500,263542.517493
12,144000,161077.422989
1248,129500,126331.202216
244,205000,242434.911439
747,265979,166348.625360


# XGBoost Regressor

In [26]:
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=70)
k_z, k_mse, k_rmse, b_z, b_mse, b_rmse = evaluate_model(xgb, f, X.values, Y.values.ravel(), k=5, B=5)

xgb.fit(X_train_pca, y_train)
xgb.score(X_test_pca, y_test.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared log Error:  0.04850286442373754
K-fold Square Root Mean Squared log Error:  0.22023365869852304
Time elapsed for k-fold:  1.6317719999999554


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  1931012805.5767913
Bootstrapping Square Root Mean Squared Error:  43943.29079139148
Time elapsed for bootstrapping:  1.836507999999867


0.7020575659284203

In [27]:
predicted = xgb.predict(X_test_pca)
xgb_pred = y_test.copy()
xgb_pred['predicted'] = predicted
xgb_pred.head()

,SalePrice,predicted
665,230500,240035.984375
12,144000,132990.859375
1248,129500,121075.304688
244,205000,205812.000000
747,265979,179845.593750


# SVM (SVR)

In [28]:
from sklearn import svm
svr_model = svm.SVR(kernel="poly", coef0=-3500, gamma="auto")
# coef0 only works with poly and sigmoid kernels
# it just puts that value instead of the column of 1's

# without it, this model breaks for some reason

k_z, k_mse, k_rmse, b_z, b_mse, b_rmse = evaluate_model(svr_model, f, X.values, Y.values.ravel(), k=5, B=5)

# epsilon, degree
svr_model.fit(X_train_pca, y_train.values.ravel())
svr_model.score(X_test_pca, y_test.values.ravel())

Evaluating K-fold with 5 folds.
K-fold Mean Squared log Error:  0.054210540665989494
K-fold Square Root Mean Squared log Error:  0.2328315714545377
Time elapsed for k-fold:  1.4187540000000354


Evaluating bootstrapping with 5 bootstraps.
Bootstrapping Mean Squared Error:  2794936853.206544
Bootstrapping Square Root Mean Squared Error:  52867.16233359366
Time elapsed for bootstrapping:  2.0181800000000294


0.7435647066294715

In [29]:
svr_predicted = svr_model.predict(X_test_pca)
svr_pred = y_test.copy()
svr_pred["predicted"] = svr_predicted
svr_pred.head()

,SalePrice,predicted
665,230500,218273.537409
12,144000,129057.076845
1248,129500,118079.296060
244,205000,210988.836908
747,265979,151240.460527
